# Run a UiPath Job using Python.
Orchestrator API 및 Python을 사용하여 작업(Job)을 실행하는 방법을 4 단계로 설명합니다. 
Orchestrator의 응답은 JSON 형식이므로 Python의 JSON 패키지를 사용하여 구문을 분석했습니다.

In [1]:
import requests
import json
import pandas as pd
from io import StringIO

## 1 단계 : 인증 토큰 받기.
Orchestrator는 서버가 사용자를 식별하기 위해 발행하는 비밀 값에 불과한 임시 베어러 토큰(bearer tokens)을 발행합니다. 이들은 수명이 짧으므로 정기적으로 재생성해야합니다.(24시간 유효)
토큰을 생성하려면 오케 스트레이터가 사용자를 식별하고 인증 할 수 있도록 몇 가지 정보를 제공해야합니다.
```
user_data = """{
    	"tenancyName": "Your_Tenant_Name",
    	"usernameOrEmailAddress": "Your_Email_ID@email.com",
    	"password": "Your_Password"
    }"""
```
위에 표시된 것은 Bearer 토큰을 생성하는 데 필요한 최소 정보입니다. 위의 정보를 사용하여 다음 방법으로 전달하십시오.


In [2]:
user_data = """{
    "tenancyName": "Default",
    "usernameOrEmailAddress": "admin",
    "password": "1q2w3e4r!"
}"""
#     "tenancyName": "Default",

Base_URL="https://koreascoring.iptime.org"
path="/api/Account/Authenticate"
URL=Base_URL + path

In [3]:
def getUserToken(URL, user_data):
    data = requests.post(URL, json=json.loads(user_data), verify=False)
    authentication_data = json.loads(data.text)
    token = "Bearer " + str(authentication_data["result"])
    return token


In [4]:
token = getUserToken(URL,user_data)
token

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'Bearer eyJhbGciOiJSUzI1NiIsImtpZCI6IjBERDBEODAzNDk3QTMwMkM1NUMwQjA0MzlBQjgzQzk4QkE4QjIwMDEiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJEZERZQTBsNk1DeFZ3TEJEbXJnOG1McUxJQUUifQ.eyJuYmYiOjE2MTk3NDYxODcsImV4cCI6MTYxOTc1MDA4NywiaXNzIjoiaHR0cHM6Ly9rb3JlYXNjb3JpbmcuaXB0aW1lLm9yZy9pZGVudGl0eSIsImF1ZCI6Ik9yY2hlc3RyYXRvckFwaVVzZXJBY2Nlc3MiLCJjbGllbnRfaWQiOiI0ZDdmMmJmMi05MDkzLTQ5NTktOWQ1Mi1iNGE2NjBmMTQ4NWYiLCJzdWIiOiI2MTM4MGJhMy1iNjc1LTQ3NzUtYTEyOC05MTk4NzcyMmU0MjUiLCJhdXRoX3RpbWUiOjE2MTk3NDY0ODYsImlkcCI6ImxvY2FsIiwiQXNwTmV0LklkZW50aXR5LlNlY3VyaXR5U3RhbXAiOiJWWEQ3SVZOQzdZUkQyV0VVNk5QQzVJR1haNzdCM1pTUyIsInBydF9pZCI6ImE1ZDk4NTBhLTE5NTgtNDYyMC05YWNhLTI5MjZlNzJlYTM4YSIsImhvc3QiOiJGYWxzZSIsImZpcnN0X25hbWUiOiIiLCJsYXN0X25hbWUiOiIiLCJwcnRfYWRtIjoiRmFsc2UiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhZG1pbiIsIm5hbWUiOiJhZG1pbiIsImVtYWlsIjoiYWRtaW5AZGVmYXVsdHRlbmFudC5jb20iLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsInNjb3BlIjpbImVtYWlsIiwib3BlbmlkIiwicHJvZmlsZSIsInRlbmFudCIsIk9yY2hlc3RyYXRvckFwaVVzZXJBY2Nlc3MiXSwiYW1yIjpbInB3ZCJdfQ.Eci3rIyQH1gsIV

## Orchestrator에등록된 프로세스 확인하기.
/odata/Releases를 이용하여 등록된 프로세스 목록을 가져온다.

In [5]:
def getReleases(token):
#     process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
#                                 headers={"Authorization": token}, verify=False)
    process_data = requests.get(f"https://koreascoring.iptime.org/odata/Releases",
                                headers={"Authorization": token}, verify=False)
    process_json = json.loads(process_data.text)
    return process_json

In [6]:
releases_data = getReleases(token)
# 판다스이용 테이블 자료로 변환 
table_data = pd.DataFrame(releases_data['value'])
table_data

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,Key,ProcessKey,ProcessVersion,IsLatestVersion,IsProcessDeleted,Description,Name,EnvironmentId,EnvironmentName,InputArguments,...,RequiresUserInteraction,AutoUpdate,FeedId,JobPriority,CreationTime,OrganizationUnitId,OrganizationUnitFullyQualifiedName,Id,Arguments,ProcessSettings
0,da8537d3-7dd5-4926-974f-7eb79b3d39ea,쇼핑몰상품이름,1.0.1,False,False,쇼핑몰 페이지를 접속하지 않고 원하는 상품 이름 가져오기,쇼핑몰상품이름_DemoProject,2,DemoProject,None,...,True,False,6f7ac8e9-9d48-4111-811c-4a9247bd938e,Normal,2020-10-20T05:44:17.527Z,1,Default,19,"{'Input': None, 'Output': None}",None
1,033f2564-164c-4ce9-baa8-5b65a0d433b2,Orchestrator_API_Test,1.0.3,False,False,Blank Process,Orchestrator_API_Test_BotWithCognigy,5,BotWithCognigy,"{""in_Arg1"":""RPA""}",...,True,False,6f7ac8e9-9d48-4111-811c-4a9247bd938e,Normal,2021-04-28T02:31:17.613Z,1,Default,33,"{'Input': '[{""name"":""in_Arg1"",""type"":""System.S...",None


In [7]:
table_data[['Key','ProcessKey','InputArguments']]

,Key,ProcessKey,InputArguments
0,da8537d3-7dd5-4926-974f-7eb79b3d39ea,쇼핑몰상품이름,None
1,033f2564-164c-4ce9-baa8-5b65a0d433b2,Orchestrator_API_Test,"{""in_Arg1"":""RPA""}"


In [8]:
json_list = json.loads(json.dumps(list(table_data[['Key','ProcessKey','InputArguments']].T.to_dict().values())))

In [9]:
json_list

[{'Key': 'da8537d3-7dd5-4926-974f-7eb79b3d39ea',
  'ProcessKey': '쇼핑몰상품이름',
  'InputArguments': None},
 {'Key': '033f2564-164c-4ce9-baa8-5b65a0d433b2',
  'ProcessKey': 'Orchestrator_API_Test',
  'InputArguments': '{"in_Arg1":"RPA"}'}]

In [22]:
json_list[1]['InputArguments']

'{"in_Arg1":"RPA"}'

## Orchestrator에등록된 로봇 확인하기.
/odata/Robots를 이용하여 등록된 로봇 목록을 가져온다.

In [10]:
def getRobots(token):
#     process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
#                                 headers={"Authorization": token}, verify=False)
    robots_data = requests.get(f"https://koreascoring.iptime.org/odata/Robots",
                                headers={"Authorization": token}, verify=False)
    robots_json = json.loads(robots_data.text)
    return robots_json

In [11]:
robots_data = getRobots(token)
# 판다스이용 테이블 자료로 변환 
robots_table_data = pd.DataFrame(robots_data['value'])
robots_table_data

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,LicenseKey,MachineName,MachineId,Name,Username,ExternalName,Description,Version,Type,HostingType,...,Password,CredentialStoreId,UserId,Enabled,CredentialType,RobotEnvironments,IsExternalLicensed,LimitConcurrentExecution,Id,ExecutionSettings
0,None,DESKTOP-2L90K0T,3,baebae,DESKTOP-2L90K0T\sjkin,None,bae pro,None,NonProduction,Standard,...,None,None,9,False,None,DemoProject,False,False,3,None
1,None,CHATBOT-DEV,10,CHATBOT-DEV,chatbot-dev\koreascoring,None,None,None,NonProduction,Standard,...,None,None,28,True,None,BotWithCognigy,False,False,11,None


## Orchestrator에 처리된 작업(프로세스 실행 결과, 로봇작업결과) 확인하기.
/odata/Jobs를 이용하여 등록된 로봇 목록을 가져온다.

In [12]:
def getJobs(token):
#     process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
#                                 headers={"Authorization": token}, verify=False)
    jobs_data = requests.get(f"https://koreascoring.iptime.org/odata/Jobs",
                                headers={"Authorization": token}, verify=False)
    jobs_json = json.loads(jobs_data.text)
    return jobs_json

In [13]:
jobs_data = getJobs(token)
# 판다스이용 테이블 자료로 변환 
jobs_table_data = pd.DataFrame(jobs_data['value'])
jobs_table_data

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,Key,StartTime,EndTime,State,JobPriority,Source,SourceType,BatchExecutionKey,Info,CreationTime,...,ResumeVersion,StopStrategy,RuntimeType,RequiresUserInteraction,ReleaseVersionId,EntryPointPath,OrganizationUnitId,OrganizationUnitFullyQualifiedName,Reference,Id
0,4256c127-4f68-4875-8005-400311afb291,2020-10-20T06:31:58.743Z,2020-10-20T06:32:07.623Z,Successful,Normal,Manual,Manual,5f886eaa-f727-46aa-8004-39424cbb9dd2,작업이 완료되었습니다.,2020-10-20T06:31:58.197Z,...,None,None,NonProduction,True,19.0,None,1,Default,,183
1,be4e69cd-1b7f-44be-91d0-5b09ab742978,2020-10-20T23:10:00.39Z,2020-10-20T23:10:06.37Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,5a02d2b5-88d7-4407-a257-fdf97e76a96d,작업이 완료되었습니다.,2020-10-20T23:10:00.183Z,...,None,None,NonProduction,True,19.0,None,1,Default,,191
2,c3c97b5a-8b8e-45cd-8f4b-3b1a88b398bf,2020-10-21T23:10:01.253Z,2020-10-21T23:10:07.41Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,7acf77a9-4efa-44db-955c-f681aa82a288,작업이 완료되었습니다.,2020-10-21T23:10:00.707Z,...,None,None,NonProduction,True,19.0,None,1,Default,,192
3,36d93d16-acab-483e-923b-86a082920821,2020-10-22T23:10:01.483Z,2020-10-22T23:10:07.703Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,08f7d08c-41d6-4649-856e-34815365ff15,작업이 완료되었습니다.,2020-10-22T23:10:00.793Z,...,None,None,NonProduction,True,19.0,None,1,Default,,193
4,2e3f9160-bc5b-439c-ad7d-0a62fc800672,2020-10-28T08:38:02.487Z,2020-10-28T08:38:02.487Z,Stopped,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,7349f2f0-cbd7-44cc-bafa-a6a42d1b4c8d,None,2020-10-23T23:10:00.83Z,...,None,Kill,None,True,NaN,None,1,Default,,194
5,246e9805-2d8e-4d3b-87a4-4f8e7cff4932,2020-10-28T23:10:01.037Z,2020-10-28T23:10:10.457Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,f3716e7d-a901-4e42-8c30-9dc4f30c55f3,작업이 완료되었습니다.,2020-10-28T23:10:00.457Z,...,None,None,NonProduction,True,19.0,None,1,Default,,195
6,b5c456d0-8874-4375-9998-002af64ed01d,2020-10-29T23:10:01.55Z,2020-10-29T23:10:09.507Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,66d863e7-08ed-474b-ac9b-99c08058c8a4,작업이 완료되었습니다.,2020-10-29T23:10:00.817Z,...,None,None,NonProduction,True,19.0,None,1,Default,,196
7,80218cae-3b64-4f67-8224-cc6d94f07164,2020-10-30T23:10:01.51Z,2020-10-30T23:10:10.053Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,1ed860d1-7c36-4acd-9cbf-c556f123d9c8,작업이 완료되었습니다.,2020-10-30T23:10:00.85Z,...,None,None,NonProduction,True,19.0,None,1,Default,,198
8,053eb2f3-de48-4563-9bbd-35426f407639,2020-10-31T23:10:01.413Z,2020-10-31T23:10:11.61Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,6b135db8-9bcd-4893-afa2-10d885d635e1,작업이 완료되었습니다.,2020-10-31T23:10:00.773Z,...,None,None,NonProduction,True,19.0,None,1,Default,,199
9,6f203273-db59-4958-82f6-1082cbb80505,2020-11-01T23:10:01.597Z,2020-11-01T23:10:10.783Z,Successful,Normal,(08시10분)쇼핑몰상품이름검색,Schedule,0b19d9f4-a479-4843-96d1-a2f15ca2d0ef,작업이 완료되었습니다.,2020-11-01T23:10:00.767Z,...,None,None,NonProduction,True,19.0,None,1,Default,,200


## 2 단계 : Process  ID를 가져옵니다.
Process ID (릴리스 키(Release Key)라고도 함)는 오케 스트레이터에서 프로세스를 식별하는 고유 한 방법입니다. 프로세스의 이름을 사용하여 ID를 식별하고 검색합니다. 여기서 1 단계에서 생성 된 토큰과 함께 프로세스 이름을 전달하면됩니다.

In [14]:
def getProcessID(process_name, token):
#     process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
#                                 headers={"Authorization": token}, verify=False)
    process_data = requests.get(f"https://koreascoring.iptime.org/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
                                headers={"Authorization": token}, verify=False)
    process_json = json.loads(process_data.text)
    # print(json.dumps(process_json, indent=2))
    process_ID = process_json["value"][0]["Key"]
    return process_ID
#     return process_json


In [15]:
# process_name="쇼핑몰상품이름"
process_name="Orchestrator_API_Test" # ProcessKey = "Orchestrator_API_Test"
getProcessID(process_name,token)

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'033f2564-164c-4ce9-baa8-5b65a0d433b2'

## 3 단계 : Robot ID를받습니다.
이제 우리는 프로세스를 실행할 로봇을 식별해야합니다. 이를 위해 로봇의 이름을 사용합니다.

In [16]:
def getRobotID(robotName, token):
#     robot_name = requests.get(f"https://platform.uipath.com/odata/Robots?$filter=Name%20eq%20'{robotName}'",
#                               headers={"Authorization": token})
    robot_name = requests.get(f"https://koreascoring.iptime.org/odata/Robots?$filter=Name%20eq%20'{robotName}'",
                              headers={"Authorization": token}, verify=False)
    robot_json_obj = json.loads(robot_name.text)
    robot_ID = robot_json_obj["value"][0]["Id"]
    return robot_ID


In [17]:
robotName="CHATBOT-DEV"
getRobotID(robotName,token)

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


11

## 4 단계 : 봇을 실행합니다.
이제 모든 정보를 수집 한 후 다음 코드 스 니펫을 사용하여 봇을 실행하기 만하면됩니다.

In [18]:
def runJob(process_ID, robot_ID, token):
    start_job_json = """{ "startInfo":
       { "ReleaseKey": \"""" + process_ID + """\",
         "Strategy": "Specific",
         "RobotIds": [ """ + str(robot_ID) + """ ],
         "Source": "Manual",
         "InputArguments": "{'in_Arg1':'안녕 챗봇 테스트'}"        
       } 
    }"""  # InputArguments should be left {} or not included if workflow does not accept any input
    start_job_data = requests.post("https://koreascoring.iptime.org/odata/Jobs/UiPath.Server.Configuration.OData.StartJobs",
                                   json=json.loads(start_job_json), headers={"Authorization": token}, verify=False)
    return start_job_data

In [19]:
process_ID='033f2564-164c-4ce9-baa8-5b65a0d433b2'
robot_ID='11'
runJob(process_ID, robot_ID, token)

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<Response [201]>

## 5 단계 : 봇 실행 결과를 확인합니다.
 ‘/odata/Jobs({0})’로 작업 결과를 가져와 봇을 실행후 수행 결과를 확인합니다.

In [20]:
def getJobResult(token):
#     process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
#                                 headers={"Authorization": token}, verify=False)
    jobresult_data = requests.get(f"https://koreascoring.iptime.org/odata/Jobs({0})",
                                headers={"Authorization": token}, verify=False)
    jobresult_json = json.loads(jobresult_data.text)
    return jobresult_json

In [21]:
getJobResult(token)

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{'message': 'Job이(가) 없습니다.', 'errorCode': 1002, 'resourceIds': None}

#### 여기서 주목할 몇 가지 사항-
##### 전략(Strategy) :
이 프로세스(process)를 실행하는 방법을 정의합니다. 'Specific'은 특정 로봇(a specific robot)에서이 프로세스를 실행하고자 함을 의미합니다. 'Dynamic'은 환경(environment)에서 사용 가능한 모든 봇(the available bots)에서 프로세스를 실행하고자 함을 의미합니다.

##### InputArguments :
프로세스의 입력 인수를 정의합니다. 프로세스에 인수가 없으면 비워 둡니다.

#### 코드 및 결론.
그래서 이것이 파이썬을 통해 Orchestrator API를 사용하여 봇을 실행하는 방법입니다.
전체 코드는 아래에 있습니다.

In [ ]:
import requests
import json


def getUserToken(URL, user_data):
    data = requests.post(url, json=json.loads(user_data))
    authentication_data = json.loads(data.text)
    token = "Bearer " + str(authentication_data["result"])
    return token


def getProcessID(process_name, token):
    process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
                                headers={"Authorization": token})
    process_json = json.loads(process_data.text)
    # print(json.dumps(process_json, indent=2))
    process_ID = process_json["value"][0]["Key"]
    return process_ID


def getRobotID(robotName, token):
    robot_name = requests.get(f"https://platform.uipath.com/odata/Robots?$filter=Name%20eq%20'{robotName}'",
                              headers={"Authorization": token})
    robot_json_obj = json.loads(robot_name.text)
    robot_ID = robot_json_obj["value"][0]["Id"]
    return robot_ID


def runJob(process_ID, robot_ID, token):
    start_job_json = """{ "startInfo":
       { "ReleaseKey": \"""" + process_ID + """\",
         "Strategy": "Specific",
         "RobotIds": [ """ + str(robot_ID) + """ ],
         "Source": "Manual",
         "InputArguments": "{'in_Arg1':'Aloha'}"        
       } 
    }"""  # InputArguments should be left {} or not included if workflow does not accept any input
    start_job_data = requests.post("https://platform.uipath.com/odata/Jobs/UiPath.Server.Configuration.OData.StartJobs",
                                   json=json.loads(start_job_json), headers={"Authorization": token})
    return start_job_data



if __name__ == "__main__":
    # ---Steps to run a JOB---

    # ---Step 1. - Get the Orchestrator TOKEN---
    # Authentication Data
    
    tenant_name = str(input("Please enter the Tenant Name: \n"))
    owner_name = str(input("Please enter the Tenant Name: \n"))
    tenant_email  = str(input("Please enter the Tenant Email: \n"))
    tenant_password  = str(input("Please enter the Tenant Password: \n"))
    url = furl = r"https://platform.uipath.com/{owner_name}/{tenant_name}/api/Account/Authenticate"
    user_data = """{
        	"tenancyName": \"""" + tenant_name + """\",
        	"usernameOrEmailAddress": \"""" + tenant_email +"""\",
        	"password": \""""+ tenant_password + """\"
        }"""
    token = getUserToken(url, user_data)
    print("Token = " + token)

    # ---Step 2. - Get PROCSS_ID of process you want to run---
    process_name = str(input("Please enter the Process Name: \n"))
    process_ID = getProcessID(process_name, token)
    print(f"Process ID = {process_ID}")

    # ---Step 3. - Get the ROBOT_ID of robot you want to run process on---
    robot_name = str(input("Please enter the Robot Name: \n"))
    robot_ID = getRobotID(robot_name, token)
    print(f"Robot ID = {robot_ID}")

    # ---Step 4. - START_Job---
    start_job_data = runJob(process_ID, robot_ID, token)
    print("Job Status = " + str(start_job_data))